In [1]:
import os

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'

os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [72]:
def get_timespan(df, num_days, max_days,future_date):
    feature = []
    for i in range(max_days,len(df)-future_date):
        temp_sum = 0
        for j in range(i-num_days,i):
            temp_sum+=df['Price'][j]
        feature.append(temp_sum/num_days)
    return np.array(feature)

def get_dow(df,num_days,max_days,future_date):
    feature = []
    for i in range(max_days, len(df) - future_date):
        temp_sum = 0
        for j in range(i - num_days*7, i,7):
            temp_sum += df['Price'][j-5] + df['Price'][j-6] + df['Price'][j-4]
        feature.append(temp_sum / (num_days*3))
    return np.array(feature)

In [128]:
df = pd.read_csv('a.csv')
X = pd.DataFrame({
    "day_1": get_timespan(df,1,140,30),
    "mean_3": get_timespan(df,3,140,30),
    "mean_7": get_timespan(df,7,140,30),
    "mean_14": get_timespan(df,14,140,30),
    "mean_30": get_timespan(df,30,140,30),
    "mean_60": get_timespan(df,60,140,30),
    "mean_140": get_timespan(df,140,140,30)})
y = df['Price'][170:]
X.head()

,day_1,mean_14,mean_140,mean_3,mean_30,mean_60,mean_7
0,79.579994,74.420846,78.011435,78.094498,71.518402,74.044834,76.260641
1,81.111931,75.082625,78.096455,79.416295,71.862171,74.103468,77.183151
2,81.478439,75.745451,78.206393,80.723455,72.221089,74.184072,78.048073
3,79.616631,76.346496,78.294624,80.735667,72.594422,74.219615,78.818745
4,79.638626,76.943353,78.350582,80.244565,72.893723,74.263333,79.447017


In [140]:
from sklearn.metrics import mean_squared_error
from math import sqrt
t_err = 0
min_err = 100
best_params = []
for eta in [.1,.3,.5,.7,.9]:
    for mdepth in [2,3,5]:
        for min_cweight in [15,20,25,30,50,60]:            
            param = {}
            param['objective'] = 'reg:linear'
            param['eta'] = eta
            param['max_depth'] = mdepth
            param['silent'] = 1
            param['eval_metric'] = 'rmse'
            param['min_child_weight'] = min_cweight
            param['subsample'] = 0.8
            param['colsample_bytree'] = 0.7
            param['seed'] = 137
            plst = list(param.items())

            num_rounds = 100
            dtrain = xgb.DMatrix(X[:len(X)-150],label=y[:len(y)-150])
            dval = xgb.DMatrix(X[len(X)-150:], label=y[len(y)-150:])
            watchlist = [(dtrain,'train'),(dval,'val')]
            model = xgb.train(plst,dtrain,num_rounds,watchlist,early_stopping_rounds=50,verbose_eval=50)
            val_pred = model.predict(dval)
            if min_err > mean_squared_error(y[len(y)-150:],val_pred):
                min_err = mean_squared_error(y[len(y)-150:],val_pred)
                t_err = mean_squared_error(y[:len(y)-150],model.predict(dtrain))
                best_params = param
print("Validation rmse:", sqrt(min_err))
print("Training rmse:", sqrt(t_err))
print(best_params)

[0]	train-rmse:58.0529	val-rmse:54.3381
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 rounds.
[50]	train-rmse:3.021	val-rmse:2.95084
Stopping. Best iteration:
[31]	train-rmse:4.17762	val-rmse:1.96161

[0]	train-rmse:58.0529	val-rmse:54.3381
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 rounds.
[50]	train-rmse:3.03333	val-rmse:2.93478
Stopping. Best iteration:
[31]	train-rmse:4.17917	val-rmse:1.96167

[0]	train-rmse:58.0529	val-rmse:54.3381
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 rounds.
[50]	train-rmse:3.00795	val-rmse:3.05152
Stopping. Best iteration:
[31]	train-rmse:4.18462	val-rmse:1.9748

[0]	train-rmse:58.0529	val-rmse:54.3381
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will tra

[50]	train-rmse:2.06031	val-rmse:2.72284
Stopping. Best iteration:
[9]	train-rmse:3.86973	val-rmse:2.28037

[0]	train-rmse:45.3604	val-rmse:42.2671
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 rounds.
[50]	train-rmse:2.2451	val-rmse:2.9188
Stopping. Best iteration:
[8]	train-rmse:4.42666	val-rmse:2.6817

[0]	train-rmse:45.3522	val-rmse:42.2638
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 rounds.
[50]	train-rmse:0.892796	val-rmse:3.01566
Stopping. Best iteration:
[9]	train-rmse:2.90743	val-rmse:1.87184

[0]	train-rmse:45.3522	val-rmse:42.2638
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.05358	val-rmse:3.16698
Stopping. Best iteration:
[7]	train-rmse:4.71875	val-rmse:2.06509

[0]	train-rmse:45.3522	val-

Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.66639	val-rmse:2.97091
Stopping. Best iteration:
[2]	train-rmse:4.75913	val-rmse:2.41996

[0]	train-rmse:20.222	val-rmse:18.1766
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 rounds.
[50]	train-rmse:2.01973	val-rmse:2.70405
Stopping. Best iteration:
[2]	train-rmse:4.79037	val-rmse:2.42176

[0]	train-rmse:20.2472	val-rmse:18.1797
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 rounds.
[50]	train-rmse:2.00277	val-rmse:3.09838
Stopping. Best iteration:
[2]	train-rmse:4.83324	val-rmse:2.46765

[0]	train-rmse:20.2122	val-rmse:18.1732
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 r

[50]	train-rmse:0.787324	val-rmse:4.09303
Stopping. Best iteration:
[1]	train-rmse:3.39676	val-rmse:3.91811

[0]	train-rmse:8.57752	val-rmse:6.37963
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 rounds.
[50]	train-rmse:0.871037	val-rmse:2.85838
Stopping. Best iteration:
[36]	train-rmse:1.04305	val-rmse:2.68516

[0]	train-rmse:8.57752	val-rmse:6.37963
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 rounds.
[50]	train-rmse:0.96354	val-rmse:3.29321
Stopping. Best iteration:
[1]	train-rmse:3.729	val-rmse:2.47715

[0]	train-rmse:8.60251	val-rmse:6.38106
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.16955	val-rmse:2.98995
Stopping. Best iteration:
[8]	train-rmse:2.60691	val-rmse:2.42751

[0]	train-rmse:8.67098	v

In [141]:
plst = list(best_params.items())
watchlist = [(dtrain,'train'),(dval,'val')]
model = xgb.train(plst,dtrain,num_rounds,watchlist,early_stopping_rounds=50,verbose_eval=50)
val_pred = model.predict(dval)
print(val_pred[len(val_pred)-30:])

[0]	train-rmse:20.2122	val-rmse:18.1732
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.0407	val-rmse:2.14106
Stopping. Best iteration:
[6]	train-rmse:2.50458	val-rmse:1.96219

[ 60.17632675  62.69451904  62.01259995  61.3188858   61.72420883
  62.11746597  62.11746597  62.67782211  62.67782211  62.67782211
  63.0185318   62.22187042  61.31410217  61.27118683  58.35736847
  58.653965    58.64599228  61.94524765  61.95322037  60.35641098
  59.79175568  58.78505707  58.78505707  58.78505707  59.59955215
  59.461689    58.72303772  58.72303772  58.40110779  58.67012024]


In [133]:
plst = list(best_params.items())
watchlist = [(dtrain,'train')]
num_rounds = 100
dtrain = xgb.DMatrix(X,label=y)
model = xgb.train(plst,dtrain,num_rounds,watchlist,early_stopping_rounds=50,verbose_eval=10)

[0]	train-rmse:45.4358
Will train until train-rmse hasn't improved in 50 rounds.
[10]	train-rmse:3.17492
[20]	train-rmse:2.16637
[30]	train-rmse:1.85612
[40]	train-rmse:1.63553
[50]	train-rmse:1.46808
[60]	train-rmse:1.34361
[70]	train-rmse:1.23683
[80]	train-rmse:1.18374
[90]	train-rmse:1.10988
[99]	train-rmse:1.0738


In [138]:
hidden_x = pd.DataFrame({
    "day_1": get_timespan(df,1,145,0),
    "mean_3": get_timespan(df,3,145,0),
    "mean_7": get_timespan(df,7,145,0),
    "mean_14": get_timespan(df,14,145,0),
    "mean_30": get_timespan(df,30,145,0),
    "mean_60": get_timespan(df,60,145,0),
    "mean_140": get_timespan(df,140,145,0)})
dtest = xgb.DMatrix(hidden_x)
val_test=model.predict(dtest)
print(val_test[len(val_test)-30:])

[ 62.37982559  62.35004807  61.72863007  62.90294266  62.01059723
  62.3216095   62.61383057  62.95279312  62.95279312  63.05638504
  63.13244629  63.1162262   62.80521393  62.80521393  62.80521393
  62.55361557  62.55361557  62.2613945   62.2613945   62.2613945
  62.2613945   62.10736084  62.07033539  61.51049805  61.51049805
  61.51049805  61.80271149  61.76037979  61.76037979  62.00146866]
